In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [5]:
# Парсер на основе связки pandas, requests, bs4

def hacker_news(pages=10):

    news = pd.DataFrame(columns=['news', 'points'])

    # Функция расчета времени
    def get_load_time(article_url):
        # будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
        try:

            # делаем запрос по url статьи article_url
            response = requests.get(
                article_url, stream=True, timeout=3.000
            )
            # получаем время загрузки страницы
            load_time = response.elapsed.total_seconds()
        except Exception as e:
            print(e)
            load_time = ">3"
        return load_time

    # Функция получения id, news_headline, rating, link
    def get_info(page=1):
        url = f'https://news.ycombinator.com/?p={page}'

        responce = requests.get(url)

        # Получаем заголовок
        page = pd.read_html(responce.text)[2]
        page.drop([1], axis=1, inplace=True)
        page.rename(columns={0: 'index', 2: 'news'}, inplace=True)

        page = page[(~page['news'].isna()) & (page['news'] != 'More')]

        # Получаем рейтинг
        points = pd.DataFrame(
            page['news'][page['index'].isna()].reset_index(drop=True))
        points.rename(columns={'news': 'points'}, inplace=True)

        # Проверяем есть ли у новости рейтинг, если его нет то присваиваем ноль
        points['contains_points'] = points['points'].str.contains("points")
        points['points'] = points['points'].str.split(' ')

        def set_points(row):
            if row['contains_points']:
                return row[0][0]
            else:
                return 0

        points['points'] = points.apply(set_points, axis=1)
        
        
        # Получаем колличество комментариев
        comments = pd.DataFrame(
            page['news'][page['index'].isna()].reset_index(drop=True))
        comments.rename(columns={'news': 'comments'}, inplace=True)
        
        comments['n_comments'] = comments['comments'].str.split('|')
        comments['contains_comments'] = comments['comments'].str.contains("comment*")
        comments['n_comments'] = comments['n_comments'].apply(lambda x: x[-1])


        def set_comments(row):
            if row['contains_comments']:
                return row[1]
            else:
                return '0 comments'

        # Чистим на отсутвие 
        comments['n_comments'] = comments.apply(set_comments, axis=1)

        # Прикручиваем рейтинг к новости
        news = pd.DataFrame(
            page['news'][~page['index'].isna()].reset_index(drop=True))

        result = pd.concat([news, points['points'], comments['n_comments']], axis=1)
        
        # Начинаем собирать ссылки и id
        soup = BeautifulSoup(responce.text, 'html.parser')

        link_list = []

        for i in range(len(soup.find_all(class_='title'))):
            try:
                link = soup.find_all(class_='title')[i].find('a')['href']
                text = soup.find_all(class_='title')[i].text

                if text != 'More':
                    link_list.append([text, link])
                else:
                    pass
            except:
                pass
        
        # Получаем id новостей
        trs = soup.find_all('tr', class_='athing')
        id_list = []

        for tr in trs:
            tr_id = tr['id']
            id_list.append(tr_id)

        id = pd.DataFrame(id_list, columns=['id'])
        
        # Получаем ссылки новостей
        links = pd.DataFrame(link_list)
        links = pd.concat([id, links], axis=1)

        links.rename(columns={0: 'news', 1: 'link'}, inplace=True)

        # Прикручиваем ссылки
        result = pd.merge(left=result, right=links)

        # Расчитываем время
        result['load_time'] = result['link'].apply(get_load_time)

        return result

    for i in range(pages):
        news = pd.concat([news, get_info(i+1)])

    news.reset_index(drop=True, inplace=True)

    return news


test = hacker_news(10)

test

HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /copernicani/status/1721981602753085582 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x136b076d0>, 'Connection to twitter.com timed out. (connect timeout=3.0)'))
HTTPSConnectionPool(host='www.bbc.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.telegraph.co.uk', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.bbc.com', port=443): Read timed out. (read timeout=3.0)
Invalid URL 'item?id=38184195': No scheme supplied. Perhaps you meant https://item?id=38184195?
Invalid URL 'item?id=38182965': No scheme supplied. Perhaps you meant https://item?id=38182965?
Invalid URL 'item?id=38161369': No scheme supplied. Perhaps you meant https://item?id=38161369?
HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /soroushjp/status/1721950722626077067 (Caused by ConnectTimeoutError(<urllib3.connection

,news,points,n_comments,id,link,load_time
0,Oh my poor business logic (rednafi.com),78,32 comments,38159363,https://rednafi.com/misc/oh_my_poor_business_l...,0.201004
1,Northlight technology in Alan Wake 2 (remedyga...,339,161 comments,38180846,https://www.remedygames.com/article/how-northl...,0.411865
2,"Go, Containers, and the Linux Scheduler (river...",232,90 comments,38181346,https://www.riverphillips.dev/blog/go-cfs/,0.161152
3,Perfect Dark: Recompiled (hackaday.com),27,5 comments,38159905,https://hackaday.com/2023/11/05/perfect-dark-r...,0.226509
4,Tumble Forth (hardcoded.net),71,9 comments,38184539,http://tumbleforth.hardcoded.net/,0.356289
...,...,...,...,...,...,...
295,Representations and srategies for games with i...,90,5 comments,38144772,https://www2.cs.sfu.ca/~bbart/personal/masters...,0.7972
296,Cortextual (cortextual.net),68,12 comments,38149839,https://cortextual.net/,0.161519
297,Mass producing the most expensive rice cooker ...,125,225 comments,38142586,https://www.youtube.com/watch?v=xLCwr8qG1p4,0.196764
298,Curated links to the best documentaries: over ...,17,2 comments,38156039,https://rocumentaries.com/,>3


---